In [1]:
import urllib.request
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import altair as alt
from urllib.request import Request, urlopen
import requests
import json
from shapely.geometry import MultiPoint


# Line Chart: How coronavirus impacts public work in Boston 

On February 1, 2020, the City of Boston announced its first confirmed case of COVID-19, a male in his 20s returning from Wuhan, China. He did not require hospitalization and is recovering and self-isolating at home.
* Source: https://www.boston.gov/departments/public-health-commission/coronavirus-timeline


On Sunday, March 8, 2020, BPHC announced five new presumptive positive cases of COVID-19 in Boston residents. These cases include one woman in her 30s, one woman in her 60s, one man in his 40s, one man in his 50s and one man in his 60s. They did not require hospitalization and are self-isolating at home. All five cases had a direct connection to the Biogen employee conference at the end of February.

In [2]:
df_2017=pd.read_csv("311_service_requests_2017.csv")
df_2018=pd.read_csv("311_service_requests_2018.csv")
df_2019=pd.read_csv("311_service_requests_2019.csv")
df_2020=pd.read_csv("https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpedrdx_uk.csv")


df_result=df_2019.append(df_2018) 
df_result=df_result.append(df_2017) 
df_result=df_result.append(df_2020) 

In [3]:
df_result.sample(3)

,case_enquiry_id,open_dt,target_dt,closed_dt,ontime,case_status,closure_reason,case_title,subject,reason,...,police_district,neighborhood,neighborhood_services_district,ward,precinct,location_street_name,location_zipcode,latitude,longitude,source
28851,101002012939,2017-02-14 11:31:42,2017-02-16 11:31:42,2017-02-14 13:57:56,ONTIME,Closed,Case Closed. Closed date : 2017-02-14 13:57:56...,Unshoveled Sidewalk,Public Works Department,Code Enforcement,...,A7,East Boston,1,Ward 1,0114,5 Orient Ave,2128.0,42.3900,-70.9993,Citizens Connect App
121507,101002942016,2019-06-23 10:21:12,2019-06-25 08:30:00,2019-06-24 09:37:38,ONTIME,Closed,Case Closed. Closed date : 2019-06-24 09:37:38...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,D4,South End,4,3,0307,INTERSECTION Savoy St & Harrison Ave,NaN,42.3594,-71.0587,Citizens Connect App
132346,101002154684,2017-07-07 17:23:09,2017-07-11 08:30:00,2017-07-07 19:17:03,ONTIME,Closed,Case Closed. Closed date : 2017-07-07 19:17:03...,Requests for Street Cleaning,Public Works Department,Street Cleaning,...,C11,Dorchester,8,Ward 15,1506,50 Granger St,2122.0,42.3026,-71.0576,Citizens Connect App


In [4]:
#trend across year
df_result['DateTime'] = pd.to_datetime(df_result['open_dt']).dt.strftime('%Y-%m')
df_result['Year'] = pd.to_datetime(df_result['open_dt']).dt.strftime('%Y')
df_result['Month'] = pd.to_datetime(df_result['open_dt']).dt.strftime('%m')
df_result['Month_Date'] = pd.to_datetime(df_result['open_dt']).dt.strftime('%m-%d')

In [5]:
df_result.head(3)

,case_enquiry_id,open_dt,target_dt,closed_dt,ontime,case_status,closure_reason,case_title,subject,reason,...,precinct,location_street_name,location_zipcode,latitude,longitude,source,DateTime,Year,Month,Month_Date
0,101002767856,2019-01-01 00:04:32,2019-01-03 08:30:00,2019-01-02 02:22:29,ONTIME,Closed,Case Closed. Closed date : 2019-01-02 02:22:29...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,1701,6-8 School St,2124.0,42.2965,-71.0729,Citizens Connect App,2019-01,2019,01,01-01
1,101002767857,2019-01-01 00:08:52,2019-01-03 08:30:00,2019-01-02 02:22:11,ONTIME,Closed,Case Closed. Closed date : 2019-01-02 02:22:11...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,2209,INTERSECTION Eastburn St & Mount Vernon St,NaN,42.3594,-71.0587,Citizens Connect App,2019-01,2019,01,01-01
2,101002767859,2019-01-01 00:50:00,NaN,2019-11-23 11:36:05,ONTIME,Closed,Case Closed. Closed date : 2019-11-23 11:36:05...,Catchbasin,Boston Water & Sewer Commission,Catchbasin,...,1110,195 South St,2130.0,42.3594,-71.0587,Constituent Call,2019-01,2019,01,01-01


In [10]:
df_result_new= df_result[(df_result['Month']=="01") | (df_result['Month']=="02") | (df_result['Month']=="03")]

df_result_new=df_result_new[df_result_new["subject"]!="Disability Department"]

df_result_new.head(2)

,case_enquiry_id,open_dt,target_dt,closed_dt,ontime,case_status,closure_reason,case_title,subject,reason,...,precinct,location_street_name,location_zipcode,latitude,longitude,source,DateTime,Year,Month,Month_Date
0,101002767856,2019-01-01 00:04:32,2019-01-03 08:30:00,2019-01-02 02:22:29,ONTIME,Closed,Case Closed. Closed date : 2019-01-02 02:22:29...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,1701,6-8 School St,2124.0,42.2965,-71.0729,Citizens Connect App,2019-01,2019,01,01-01
1,101002767857,2019-01-01 00:08:52,2019-01-03 08:30:00,2019-01-02 02:22:11,ONTIME,Closed,Case Closed. Closed date : 2019-01-02 02:22:11...,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,...,2209,INTERSECTION Eastburn St & Mount Vernon St,NaN,42.3594,-71.0587,Citizens Connect App,2019-01,2019,01,01-01


In [11]:
df_new=df_result_new.groupby(["Year", 'Month', "subject"])["case_enquiry_id"].count().reset_index(name="count")

df_new.head(5)

df_new.dtypes

Year       object
Month      object
subject    object
count       int64
dtype: object

In [13]:
df_new["subject"].unique()

array(['Animal Control', 'Boston Police Department',
       'Boston Water & Sewer Commission', 'Inspectional Services',
       "Mayor's 24 Hour Hotline", 'Neighborhood Services',
       'Parks & Recreation Department', 'Property Management',
       'Public Works Department', 'Transportation - Traffic Division',
       'Consumer Affairs & Licensing'], dtype=object)

In [18]:
####chart
line_selection=alt.selection_interval(bind="scales", encodings=['x'])

base=alt.Chart(df_new).mark_line(size=2.5, strokeDash=[5, 5]).encode(
    x=alt.X("Month:O"),
    y=alt.Y("count:Q"),
    opacity=alt.value(0.8),
    color=alt.Color('Year',scale=alt.Scale(domain=[2017,2018,2019], range=["#ff9896", "#9467bd", "#2ca02c"])),
).transform_filter(
    alt.FieldOneOfPredicate(field='Year', oneOf=[2017,2018,2019])).transform_filter(
    alt.FieldOneOfPredicate(field='subject', oneOf=["Inspectional Services",
                                                    'Animal Control',
                                                   "Parks & Recreation Department",
                                                   "Transportation - Traffic Division",
                                                   "Public Works Department",
                                                   "Property Management"]))



line_2020=alt.Chart(df_new).mark_line(size=2.5).encode(
    x=alt.X("Month:O"),
    y=alt.Y("count:Q"),
).transform_filter(
    alt.FieldOneOfPredicate(field='Year', oneOf=[2020])).transform_filter(
    alt.FieldOneOfPredicate(field='subject', oneOf=["Inspectional Services",
                                                    'Animal Control',
                                                   "Parks & Recreation Department",
                                                   "Transportation - Traffic Division",
                                                   "Public Works Department",
                                                   "Property Management"]))




####create selection that chooses the nearest point
selection= alt.selection(type="single", on='mouseover',nearest=True,
                        fields=['Month'], empty='none')



###transparent selector
##create a transparent selector
selectors = alt.Chart(df_new).mark_point().encode(
    x='datetime:T',
    opacity=alt.value(0),
).add_selection(
    selection
)

#Draw points
points =alt.Chart(df_new).mark_point(color="black", size=20).encode(
    y="count:Q",
    x='Month:O',
    opacity=alt.condition(selection, alt.value(1), alt.value(0)),
    tooltip=["count:Q"]
).add_selection(selection)

combined = alt.layer(base, line_2020, points, width=250, height=150, data=df_new).facet("subject:N", columns=3).transform_filter(
    alt.FieldOneOfPredicate(field='subject', oneOf=["Inspectional Services",
                                                    'Animal Control',
                                                   "Parks & Recreation Department",
                                                   "Transportation - Traffic Division",
                                                   "Public Works Department",
                                                   "Property Management"])).resolve_scale(y="independent")

combined

#final=alt.layer(base, line_2020, points, width=100, height=150).facet("subject:N").resolve_scale(y="independent")




alt.FacetChart(...)

In [21]:
combined.save('change_cases_n.json')
combined.save('change_cases.html')

Things begin to speed up on March 8th

* Goal of this chart is to help readers better compare situation in 2019 and 2020. 
* No, i don't want another line chart
* No, I don't want them to compare across time and date

In [22]:
df_result_new= df_result[(df_result['Month']=="03")| (df_result['Month']=="02")]

df_result_new=df_result_new[df_result_new["subject"]!="Disability Department"]

df_result_new.head(2)

df_new=df_result_new.groupby(["Year", "Month_Date", "subject"])["case_enquiry_id"].count().reset_index(name="count")



bars=alt.Chart(df_new).mark_bar().encode(
    x='Month_Date:O',
    y="count:Q",
    # The highlight will be set on the result of a conditional statement
    color=alt.condition(
        alt.datum.Month_Date == "03-08",  # If the year is 1810 this test returns True,
        alt.value('orange'),     # which sets the bar orange.
        alt.value('steelblue')   # And if it's not true it sets the bar steelblue.
    ),
).properties(width=600).transform_filter(
    alt.FieldOneOfPredicate(field='Year', oneOf=[2020]))

ticks=alt.Chart(df_new).mark_tick().encode(
    x='Month_Date:O',
    y="count:Q",
    # The highlight will be set on the result of a conditional statement
    color=alt.condition(
        alt.datum.Month_Date == "03-08",  # If the year is 1810 this test returns True,
        alt.value('orange'),     # which sets the bar orange.
        alt.value('red')   # And if it's not true it sets the bar steelblue.
    ),
).properties(width=600).transform_filter(
    alt.FieldOneOfPredicate(field='Year', oneOf=[2019]))


final=alt.layer(bars, ticks).transform_filter(
    alt.FieldOneOfPredicate(field='subject', oneOf=["Inspectional Services","Mayor's 24 Hour Hotline", "Parks & Recreation Department", "Pubic Works Department", "Transportation - Traffic Division"])).facet("subject:N").resolve_scale(y="independent")

final
# df_new["pct_change"]= df_new.groupby(["Year", "Month_Date", "subject"]).pct_change()

# df_new.head(4)

alt.FacetChart(...)

* Other people's examples:

https://ilpes.it/animated-stacked-bar-charts-with-d3js/demo